# EEP 555: Data Science for Power Systems (Winter 2026)
## Case Study: Solar Storm Impact Classification

Goal: build and compare **7 classification models** that predict the **impact level on the power grid** from solar storm measurements.

Workflow:
- Review the dataset columns and decide which columns are the **target**, **droppable identifiers**, and the **categorical feature**
- Drop unused columns and remove rows with missing values
- Train/test split (70% / 30%)
- One-hot encode the categorical feature and standardize all features
- Train + evaluate 7 classifiers using **weighted** Precision/Recall/F1
- For the best model (by weighted Test F1), inspect the confusion matrix and classification report


## Dataset columns

| Column | What it represents |
|---|---|
| `event_id` | Unique identifier for each solar storm (an ID) |
| `event_date` | Date of the event |
| `solar_flare_class` | Flare type: **C** (common), **M** (moderate), **X** (extreme) |
| `flare_intensity` | Numeric estimate (1–100) of flare energy |
| `geomagnetic_index_Kp` | Planetary Kp index (0–9) indicating geomagnetic storm strength |
| `solar_wind_speed` | Solar wind speed in km/s |
| `solar_wind_density` | Solar wind density (particles/cm³) |
| `flare_duration_minutes` | Flare duration in minutes |
| `power_grid_disruption` | Impact level (**0: None, 1: Minor, 2: Moderate, 3: Severe**) |


## Load the dataset

Load the CSV and take a quick look at:
- dataset shape
- column names
- the first few rows


In [1]:
import numpy as np
import pandas as pd

DATA_PATH = "solar_storm_impact_dataset.csv"  # <-- keep as-is unless your file name is different

df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'solar_storm_impact_dataset.csv'

## Set the key column names (you decide)

Before we build models, set three variables:

- `TARGET`: column name containing the labels you want to predict
- `DROP_COLS`: columns that should be removed because they are identifiers/metadata (hint: **two** columns can be dropped)
- `CAT_COL`: the categorical feature that should be one-hot encoded (hint: **one** categorical column is present)

Use the column list printed above to decide these values.


In [ ]:
# TODO: Set these three values based on the dataset columns.
# Hint: 2 columns can be dropped, and 1 categorical column exists.

TARGET = "power_grid_disruption"        # the label to predict
DROP_COLS = ["event_id", "event_date"]   # identifiers / metadata
CAT_COL = "solar_flare_class"            # categorical feature to one-hot encode

# Quick sanity checks (leave these as-is)
assert isinstance(TARGET, str) and TARGET in df.columns, "TARGET must be a column name in df"
assert isinstance(DROP_COLS, (list, tuple)) and all(c in df.columns for c in DROP_COLS), "DROP_COLS must be a list of column names in df"
assert isinstance(CAT_COL, str) and CAT_COL in df.columns, "CAT_COL must be a column name in df"


AssertionError: TARGET must be a column name in df

## Clean + Split the data

We are keeping things intentionally simple:
- Drop `DROP_COLS`
- Drop rows that contain any missing values in the remaining columns (and the target)
- Split into 70% train and 30% test (stratified)


In [3]:
from sklearn.model_selection import train_test_split

# Drop requested columns if present
df2 = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")

# Drop rows with missing values in any used column (no imputation)
use_cols = [c for c in df2.columns if c != TARGET]
df2 = df2.dropna(subset=use_cols + [TARGET]).reset_index(drop=True)

X = df2.drop(columns=[TARGET])
y = df2[TARGET].astype(int).values

print("Cleaned shape:", df2.shape)
print("Label distribution:\n", pd.Series(y).value_counts().sort_index())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

print("\nTrain shape:", X_train.shape, "Test shape:", X_test.shape)


NameError: name 'df' is not defined

## Preprocessing + models

Preprocessing steps:
- One-hot encode `CAT_COL`
- Standardize **all** features (numeric + one-hot)

Then we train and evaluate 7 classifiers:
- Logistic Regression
- Perceptron
- Linear SVM (LinearSVC)
- Gaussian Naive Bayes
- KNN
- Decision Tree
- Random Forest

We report **weighted** precision/recall/F1 (weighted by class support).


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix
)

cat_features = [CAT_COL] if CAT_COL in X.columns else []
num_features = [c for c in X.columns if c not in cat_features]

# Dense output keeps scaling straightforward
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_features),
    ],
    remainder="drop"
)

def weighted_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="weighted", zero_division=0
    )
    return acc, prec, rec, f1

models = {
    "Logistic Regression": LogisticRegression(max_iter=5000, n_jobs=-1),
    "Perceptron": Perceptron(max_iter=2000, tol=1e-3, random_state=42),
    "SVM (LinearSVC)": LinearSVC(random_state=42),
    "Naive Bayes (GaussianNB)": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=7, metric="euclidean"),
    "Decision Tree": DecisionTreeClassifier(random_state=42, max_depth=40),
    "Random Forest": RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
}


NameError: name 'X' is not defined

## Train + evaluate (tabular results)

We train each classifier once (70/30 split) and report:
- Train Accuracy / Precision / Recall / F1 (weighted)
- Test Accuracy / Precision / Recall / F1 (weighted)

Then we pick the best model based on **weighted Test F1**.


In [5]:
results = []
best = {"name": None, "pipe": None, "test_f1": -1.0}

for name, clf in models.items():
    pipe = Pipeline(steps=[
        ("prep", preprocessor),
        ("scale", StandardScaler()),
        ("clf", clf),
    ])

    pipe.fit(X_train, y_train)

    yhat_tr = pipe.predict(X_train)
    yhat_te = pipe.predict(X_test)

    tr = weighted_metrics(y_train, yhat_tr)
    te = weighted_metrics(y_test, yhat_te)

    results.append([name, *tr, *te])

    if te[3] > best["test_f1"]:
        best["name"] = name
        best["pipe"] = pipe
        best["test_f1"] = te[3]

cols = ["Model",
        "Train Acc", "Train Prec(w)", "Train Rec(w)", "Train F1(w)",
        "Test Acc",  "Test Prec(w)",  "Test Rec(w)",  "Test F1(w)"]

out = pd.DataFrame(results, columns=cols).sort_values("Test F1(w)", ascending=False)

print("\n=== Results (sorted by Test weighted F1) ===")
print(out.to_string(index=False, formatters={
    "Train Acc": "{:.4f}".format,
    "Train Prec(w)": "{:.4f}".format,
    "Train Rec(w)": "{:.4f}".format,
    "Train F1(w)": "{:.4f}".format,
    "Test Acc": "{:.4f}".format,
    "Test Prec(w)": "{:.4f}".format,
    "Test Rec(w)": "{:.4f}".format,
    "Test F1(w)": "{:.4f}".format,
}))


NameError: name 'models' is not defined

## Best model: confusion matrix + classification report

A high weighted F1 is nice, but you should still check:
- which classes are getting confused (confusion matrix)
- per-class precision/recall/F1 (classification report)

We compute these on the **test set** only.


In [6]:
print("\n" + "=" * 80)
print(f"Best model by Test weighted F1: {best['name']}  (Test F1(w)={best['test_f1']:.4f})")
print("-" * 80)

yhat_best = best["pipe"].predict(X_test)

print("\n[TEST] Classification Report:")
print(classification_report(y_test, yhat_best, digits=4, zero_division=0))

print("[TEST] Confusion Matrix (rows=true, cols=pred):")
print(confusion_matrix(y_test, yhat_best))



Best model by Test weighted F1: None  (Test F1(w)=-1.0000)
--------------------------------------------------------------------------------


AttributeError: 'NoneType' object has no attribute 'predict'